In [10]:
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min

class ConstrainedKMeans:
    def __init__(self, n_clusters, max_elements_per_cluster, max_iter=300, tol=1e-4):
        self.n_clusters = n_clusters
        self.max_elements_per_cluster = max_elements_per_cluster
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X):
        n_samples, n_features = X.shape
        self.centers = X[np.random.choice(n_samples, self.n_clusters, replace=False)]
        
        labels = np.zeros(n_samples, dtype=int)
        
        for iteration in range(self.max_iter):
            # Присвоение кластеров
            new_labels, _ = pairwise_distances_argmin_min(X, self.centers)

            # Подсчет количества элементов в каждом кластере
            cluster_counts = np.bincount(new_labels, minlength=self.n_clusters)

            # Проверка на максимальное количество элементов в каждом кластере
            for cluster_id in range(self.n_clusters):
                if cluster_counts[cluster_id] > self.max_elements_per_cluster:
                    # Получаем индексы элементов, принадлежащих текущему кластеру
                    idx_to_keep = np.where(new_labels == cluster_id)[0]
                    
                    # Случайным образом выбираем нужное количество элементов
                    if len(idx_to_keep) > self.max_elements_per_cluster:
                        idx_to_remove = np.random.choice(idx_to_keep, cluster_counts[cluster_id] - self.max_elements_per_cluster, replace=False)
                        new_labels[idx_to_remove] = -1  # Убираем элементы из этого кластера

            # Переопределение меток для неподписанных экземпляров
            unassigned_indices = np.where(new_labels == -1)[0]
            if len(unassigned_indices) > 0:
                nearest_clusters = self.predict(X[unassigned_indices])
                
                for idx, nearest in zip(unassigned_indices, nearest_clusters):
                    if cluster_counts[nearest] < self.max_elements_per_cluster:
                        new_labels[idx] = nearest
                    else:
                        # Принудительное назначение в кластер с минимальным количеством элементов
                        least_filled_cluster = np.argmin(cluster_counts)
                        new_labels[idx] = least_filled_cluster

            # Обновление меток
            labels = new_labels

            # Пересчет центров кластеров
            new_centers = np.array([X[labels == i].mean(axis=0) if np.any(labels == i) else self.centers[i] for i in range(self.n_clusters)])

            # Проверка на сходимость
            if np.linalg.norm(new_centers - self.centers) < self.tol:
                break

            self.centers = new_centers

        # Финальная проверка меток
        # Если все экземпляры не распределены, принудительно назначаем им метки
        self.labels_ = labels
        unique_labels = np.unique(self.labels_)
        
        if len(unique_labels) > self.n_clusters:
            # Удаляем лишние метки
            label_map = {}
            new_label_count = 0
            
            for i in range(len(self.labels_)):
                if self.labels_[i] not in label_map:
                    if new_label_count < self.n_clusters:
                        label_map[self.labels_[i]] = new_label_count
                        new_label_count += 1
                    else:
                        # Если превышено количество меток, назначаем последним меткам
                        label_map[self.labels_[i]] = new_label_count - 1

            # Применяем новую нумерацию меток
            self.labels_ = np.array([label_map[label] for label in self.labels_])

        return self

    def predict(self, X):
        return pairwise_distances_argmin(X, self.centers)

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Загрузка данных
# Например, данные загружаются так: [320 объектов, 10 временных шагов, 246 признаков]
data = np.load(r'../data/ts_cut/ihb.npy')

# Проверка формы данных
print("Shape of the data:", data.shape)

# 1. Заполнение пропусков (NaN) средними значениями по каждому признаку
# Например, заполняем пропуски по каждому временному шагу и признаку
data = np.nan_to_num(data, nan=np.nanmean(data, axis=0))

# 2. Нормализация данных
# Нормализуем каждый объект с учетом всех временных шагов
scaler = StandardScaler()
for i in range(data.shape[0]):  # для каждого объекта (320 объектов)
    data[i] = scaler.fit_transform(data[i])

# Проверка нормализованных данных
print("Data after normalization:", data)


C:\Users\Public\anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Shape of the data: (320, 10, 246)
Data after normalization: [[[-1.44170211e+00  7.20190967e-01 -6.77497879e-01 ...  3.83541063e-01
   -3.12533138e-01  1.87190223e-01]
  [-1.02350452e+00  3.90959972e-01  7.43412598e-01 ... -5.84490881e-01
    2.04978137e+00  5.18247925e-01]
  [-5.46366854e-01 -2.58127803e+00  4.14594983e-01 ...  3.12216153e-01
    3.78479782e-01 -1.04720349e+00]
  ...
  [ 1.14514134e+00  1.27261833e-01  1.12503640e+00 ...  1.54941084e+00
   -8.67693013e-01  9.21302912e-02]
  [-6.98623473e-03  4.01464294e-01 -5.76699984e-02 ... -2.27019587e+00
    7.81783352e-01 -1.54727588e+00]
  [ 1.41313702e+00  4.04202258e-01 -8.30078044e-01 ...  3.53709513e-01
   -9.08687734e-01 -1.69444844e+00]]

 [[ 9.60413616e-01  5.41853981e-03 -9.06809649e-01 ... -9.54797299e-01
   -3.19883294e-01 -4.63113961e-01]
  [-1.20167469e+00 -1.41383457e-01 -7.81689063e-01 ...  4.44169710e-01
   -2.58050670e+00 -1.93454718e+00]
  [-5.50830531e-01  3.15284323e-01  2.37690306e-01 ...  9.18433487e-01
    2

In [2]:
import numpy as np

# Функция для расчета корреляционной матрицы
def calculate_correlation_matrix(time_series):
    """
    time_series: массив формы (10 временных шагов, 246 признаков)
    Возвращает: корреляционная матрица формы (246, 246)
    """
    # Расчет корреляционной матрицы по временным рядам для признаков
    corr_matrix = np.corrcoef(time_series.T)  # .T для получения корреляции между признаками
    return corr_matrix

# Создание матриц корреляций для всех объектов
correlation_matrices = []

for i in range(data.shape[0]):  # 320 объектов
    corr_matrix = calculate_correlation_matrix(data[i])
    
    # Заменим NaN на 0 (в случае если корреляция не может быть рассчитана для некоторых регионов)
    corr_matrix = np.nan_to_num(corr_matrix, nan=0.0)
    
    # Добавляем матрицу в список
    correlation_matrices.append(corr_matrix)

# Преобразуем список в numpy массив
correlation_matrices = np.array(correlation_matrices)

# Проверим форму данных
print("Shape of correlation matrices:", correlation_matrices.shape)  # (320, 246, 246)

# Дополнительно: можно извлечь верхнюю треугольную часть матрицы, чтобы уменьшить размерность
def extract_upper_triangular(matrix):
    """
    Извлекает верхнюю треугольную часть корреляционной матрицы, исключая диагональ.
    """
    return matrix[np.triu_indices_from(matrix, k=1)]

# Применение функции для всех матриц
upper_triangular_matrices = np.array([extract_upper_triangular(mat) for mat in correlation_matrices])

# Проверка формы результата (например, [320 объектов, количество элементов в верхнем треугольнике])
print("Shape of upper triangular matrices:", upper_triangular_matrices.shape)  # (320, число элементов в треугольнике)


Shape of correlation matrices: (320, 246, 246)
Shape of upper triangular matrices: (320, 30135)


In [3]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
import time

# Количество кластеров соответствует количеству субъектов
n_clusters = 20

# Методы связи для экспериментов
linkage_methods = ['ward', 'complete', 'average', 'single']

# Замер времени выполнения
start_time = time.time()

for linkage_method in linkage_methods:
    print(f"Running Agglomerative Clustering with linkage='{linkage_method}'...")
    
    # Создаем модель Agglomerative Clustering с разными методами связи
    agg_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage_method)

    # Обучаем модель и предсказываем метки кластеров
    labels_agg = agg_clustering.fit_predict(upper_triangular_matrices)

    # Проверим количество уникальных меток
    unique_labels = np.unique(labels_agg)
    print(f"Unique labels from Agglomerative Clustering with linkage='{linkage_method}': {unique_labels}")

    # Сохраним предсказания в файл для сабмита
    submission_agg = pd.DataFrame({'prediction': labels_agg})
    file_name = f'submission_agg_{linkage_method}.csv'
    submission_agg.to_csv(file_name, index=False)

    print(f"Submission file '{file_name}' saved!")

# Выводим время выполнения
end_time = time.time()
print(f"Total time taken for clustering: {end_time - start_time:.2f} seconds")


Running Agglomerative Clustering with linkage='ward'...
Unique labels from Agglomerative Clustering with linkage='ward': [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Submission file 'submission_agg_ward.csv' saved!
Running Agglomerative Clustering with linkage='complete'...
Unique labels from Agglomerative Clustering with linkage='complete': [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Submission file 'submission_agg_complete.csv' saved!
Running Agglomerative Clustering with linkage='average'...
Unique labels from Agglomerative Clustering with linkage='average': [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Submission file 'submission_agg_average.csv' saved!
Running Agglomerative Clustering with linkage='single'...
Unique labels from Agglomerative Clustering with linkage='single': [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Submission file 'submission_agg_single.csv' saved!
Total time taken for clustering: 4.02 seconds

In [4]:
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Количество кластеров соответствует количеству субъектов
n_clusters = 20

# Применение PCA для уменьшения размерности до 200 компонент
pca = PCA(n_components=300)
reduced_data = pca.fit_transform(upper_triangular_matrices)

# Применение Agglomerative Clustering на данных с уменьшенной размерностью
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
labels_agg = agg_clustering.fit_predict(reduced_data)

# Сохранение результатов в файл для сабмита
submission_agg_pca = pd.DataFrame({'prediction': labels_agg})
submission_agg_pca.to_csv('submission_agg_pca_300.csv', index=False)

print("Submission file 'submission_agg_pca.csv' saved!")


Submission file 'submission_agg_pca.csv' saved!


In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Количество кластеров соответствует количеству субъектов
n_clusters = 20

# Нормализация данных с помощью MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(upper_triangular_matrices)

# Применение PCA для уменьшения размерности до 200 компонент
pca = PCA(n_components=200)
reduced_data = pca.fit_transform(scaled_data)

# Применение Agglomerative Clustering на данных с уменьшенной размерностью
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
labels_agg = agg_clustering.fit_predict(reduced_data)

# Сохранение результатов в файл для сабмита
submission_agg_pca_minmax = pd.DataFrame({'prediction': labels_agg})
submission_agg_pca_minmax.to_csv('submission_agg_pca_minmax.csv', index=False)

print("Submission file 'submission_agg_pca_minmax.csv' saved!")


Submission file 'submission_agg_pca_minmax.csv' saved!


In [6]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Количество кластеров соответствует количеству субъектов
n_clusters = 20

# Отбор признаков с использованием VarianceThreshold
selector = VarianceThreshold(threshold=0.1)  # Можно регулировать порог
selected_data = selector.fit_transform(upper_triangular_matrices)

# Применение PCA для уменьшения размерности до 200 компонент
pca = PCA(n_components=200)
reduced_data = pca.fit_transform(selected_data)

# Применение Agglomerative Clustering на данных с уменьшенной размерностью
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
labels_agg = agg_clustering.fit_predict(reduced_data)

# Сохранение результатов в файл для сабмита
submission_agg_pca_selected = pd.DataFrame({'prediction': labels_agg})
submission_agg_pca_selected.to_csv('submission_agg_pca_selected.csv', index=False)

print("Submission file 'submission_agg_pca_selected.csv' saved!")


Submission file 'submission_agg_pca_selected.csv' saved!


In [20]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Количество кластеров соответствует количеству субъектов
n_clusters = 20

# Заполнение пропусков медианой
imputer = SimpleImputer(strategy='median')
filled_data = imputer.fit_transform(upper_triangular_matrices)
print(filled_data.shape)
# Применение PCA для уменьшения размерности до 200 компонент
pca = PCA(n_components=320)
reduced_data = pca.fit_transform(filled_data)
print(reduced_data.shape)
# Применение Agglomerative Clustering на данных с уменьшенной размерностью
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
labels_agg = agg_clustering.fit_predict(reduced_data)

# Сохранение результатов в файл для сабмита
submission_agg_pca_median = pd.DataFrame({'prediction': labels_agg})
submission_agg_pca_median.to_csv('submission_agg_pca_median.csv', index=False)

print("Submission file 'submission_agg_pca_median.csv' saved!")


(320, 30135)
(320, 320)
Submission file 'submission_agg_pca_median.csv' saved!


In [11]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Количество кластеров соответствует количеству субъектов
n_clusters = 20

# Заполнение пропусков медианой
imputer = SimpleImputer(strategy='median')
filled_data = imputer.fit_transform(upper_triangular_matrices)

# Применение PCA для уменьшения размерности до 200 компонент
pca = PCA(n_components=200)
reduced_data = pca.fit_transform(filled_data)
print(reduced_data.shape)

kmeans = ConstrainedKMeans(n_clusters=20, max_elements_per_cluster=16)
kmeans.fit(reduced_data)
kmeans.labels_

# Сохранение результатов в файл для сабмита
submission_agg_pca_median = pd.DataFrame({'prediction': kmeans.labels_})
submission_agg_pca_median.to_csv('submission_kmeans_median.csv', index=False)

print("Submission file 'submission_agg_pca_median.csv' saved!")

(320, 200)
Submission file 'submission_agg_pca_median.csv' saved!
